# LBM tutorial

## installation

 - First we will assume that there is a current version of a sailfish in current directory.
 
         !git clone https://github.com/marcinofulus/sailfish.git
 
         
### on using sailfish in jupyter notebook

There are two possibilities to use sailfish in the jupyter notebook. 

1.  Create script and run from notebook:

    - we will write script in data subdirectory, using %writefile magic
    - the script will be executed by %bash magic (might be nessesary to setup conda env)
    - output will be put in data subdirectory
    - notebook is used for data processing and experimentation

2. Sailfish process in run by the python interpreter in the jupyter notebook:

    - works only in the single block i.e. `--debug_single_process` flag is mandatory
    - allows for interactive data exploration during and after after simulation, as `master` is in controller namespace
    - allows for interacting with ipywidgets during simulation
    
   

In [1]:
%%bash 
pwd

/home/users/marcin.kostur/LBM_tutorial_git


In [2]:
%%sh
cd sailfish
git pull 

Already up-to-date.


In [3]:
!git clone https://github.com/marcinofulus/sailfish

fatal: destination path 'sailfish' already exists and is not an empty directory.


### Configuration

There is a following configuration cell which:

 - tries to separate data from code
 - sets path for interactive (in jupyter) using of sailfish
 - sets `BASH_ENV` so we gain  control on python interpreter in `%%bash`  cells
 - imports matplotlib and numpy 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys, os

sys.path.append( os.path.join(os.environ.get("HOME"),"sailfish") ) 

datapath = os.path.join( "/dev/shm/",os.environ.get("USER"),"data")
try:
    os.makedirs(datapath,mode=0755)
except OSError as ex:
    print datapath,"::",ex.strerror

if not os.path.isdir('data') and os.path.islink('data'):
    os.remove("data")
    
if not os.path.islink('data'):
    os.symlink(datapath, "data")

%env BASH_ENV= ~/.bash_profile

/dev/shm/marcin.kostur/data :: File exists


#### Note:

  - In this tutorial we will use %%bash magic for running commands. If there is a need to configure e.g. conda environment, setting once in the notebook BASH_ENV variable will force all %%bash cell to run given config file.

For example we use:


In [3]:
%env BASH_ENV= ~/.bash_profile

env: BASH_ENV=~/.bash_profile


Content of this `.bash_profile` can be for  file is read by shell when `%%bash` magic is used. 
One might want to choose certain conda env:

    source activate py27


Note, that it does not work with `%%sh`.

## Checking for nvidia devices

To make sure that pycuda - required for sailfish - works one might check:

In [4]:
import pycuda.driver as cuda
cuda.init()
for i in range( cuda.Device.count()):
    device = cuda.Device(i)
    ctx = device.make_context()

    (free,total)=cuda.mem_get_info()

    print i,"::",device.name(), device.compute_capability(),\
        device.total_memory()/1024.**3,"GB", free/1024.**3,total/1024.**3
    ctx.detach()

0 :: Tesla K40m (3, 5) 11.1717529297 GB 10.5391845703 11.1717529297
1 :: Tesla K40m (3, 5) 11.1717529297 GB 11.1013183594 11.1717529297


In [ ]:
from pynvml import *
nvmlInit()

In [2]:
nvmlSystemGetDriverVersion()

'352.79'

In [3]:
deviceCount = nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvmlDeviceGetHandleByIndex(i)
    print "Device", i, ":", nvmlDeviceGetName(handle)

Device 0 : Tesla K40m
Device 1 : Tesla K40m


In [8]:
int(nvmlDeviceGetCount())
gpuObj = nvmlDeviceGetHandleByIndex(0);
print nvmlDeviceGetMemoryInfo(gpuObj).total/1024.**3,"GB"
print nvmlDeviceGetMemoryInfo(gpuObj).used/1024.**3,"GB"

11.9998168945 GB
1.780834198 GB


In [12]:
nvmlDeviceGetName(gpuObj)

'Tesla K40m'